# Converting .abf to NWB
- see `clampfit_abf_standardization_nwb.pptx`
- see Sharepoint: `Documents/data/data_standardization/pclamp_clampfit_icephys`
- see `ephys_nwb_params.xlsx`: this is maintained by experimenter
- **BE SURE TO SET**
    - **EXCEL FILE NAME** (`ephys_nwb_params.xlsx` in this example)
    - **ABF DIRECTORY** (`abf_data` in this example)

In [ ]:
import pandas as pd
from pathlib import Path
from neuroconv.datainterfaces import AbfInterface

# Path to Excel and ABF files
excel_path = Path("./ephys_nwb_params.xlsx")
ECEPHY_DATA_PATH = Path("./abf_data")
output_folder = Path("./nwb_files")
output_folder.mkdir(parents=True, exist_ok=True)

# Load Excel file
df = pd.read_excel(excel_path)
df = df.iloc[1:]  # Drop the first row after the header (provides data naming example)

# Group by EXPERIMENT ID to process multiple sessions per experiment
grouped = df.groupby("EXPERIMENT ID")

for experiment_id, group in grouped:
    # Extract shared metadata (assumed constant within experiment)
    first_row = group.iloc[0]

    # Construct icephys_metadata
    icephys_metadata = {
        "cell_id": first_row["cell_id"],
        "slice_id": first_row["slice_id"],
        "targeted_layer": first_row["targeted_layer"],
        "inferred_layer": first_row.get("inferred_layer", ""),
        "recording_sessions": [
            {
                "abf_file_name": row[".abf file"],
                "stimulus_type": row["stimulus_type"],
                "icephys_experiment_type": row["icephys_experiment_type"]
            }
            for _, row in group.iterrows()
        ]
    }

    abf_file_paths = [ECEPHY_DATA_PATH / row[".abf file"] for _, row in group.iterrows()]

    # Instantiate data interface
    interface = AbfInterface(
        file_paths=abf_file_paths,
        icephys_metadata=icephys_metadata
    )

    # Retrieve and update metadata
    metadata = interface.get_metadata()
    metadata['NWBFile'].update(
        identifier=str(experiment_id),
        session_description=first_row["session_description"],
        lab="my lab name",
        institution="My University",
        experimenter=["John Doe", "Jane Doe"]
    )
    metadata["Subject"] = {
        "subject_id": first_row["subject_id"],
        "species": first_row["species"],
        "genotype": first_row["genotype"],
        "sex": first_row["sex"],
        "date_of_birth": str(first_row["date_of_birth"])
    }

    # Run conversion
    nwb_output_path = output_folder / f"{experiment_id}.nwb"
    interface.run_conversion(nwbfile_path=nwb_output_path, metadata=metadata)

    print(f"Finished NWB conversion for experiment {experiment_id}")

/Users/pacody/Documents/code/icephys_abf2nwb/env/lib/python3.13/site-packages/pynwb/file.py:158: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)
/Users/pacody/Documents/code/icephys_abf2nwb/env/lib/python3.13/site-packages/pynwb/file.py:486: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


Finished NWB conversion for experiment 2025053001


/Users/pacody/Documents/code/icephys_abf2nwb/env/lib/python3.13/site-packages/pynwb/file.py:158: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)
/Users/pacody/Documents/code/icephys_abf2nwb/env/lib/python3.13/site-packages/pynwb/file.py:486: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


Finished NWB conversion for experiment 2025053101
